In [193]:
import random as rd
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable


In [194]:
# core logic
class RosenblattPerceptron:

    def __init__(self, theta, learning_rate, data, data_size, interval):
        self.theta = theta
        self.learning_rate = learning_rate
        self.weights = []
        self.adjustments_count = 0
        self.data = data
        self.data_size = data_size
        self.interval = interval
        self.adjusted_in_this_era = False
        self.eras_adjusting = 0

    def reset_counters(self):
        self.adjustments_count = 0
        self.eras_adjusting = 0


    def activation_degree(self, u):
        if self.theta == 1 and u >= self.theta:
            return 1
        elif self.theta == 0 and u <= self.theta:
            return 1
        else:
            return 0

    def generate_initial_weights(self):
        #plus one for the bias
        weights = []
        for j in range(self.data_size):
           weights.append(rd.uniform(self.interval[0], self.interval[1]))
        self.weights = np.array(weights)
        print("Generated initial weights! Values: {}".format(self.weights))

    def add_bias_to_data(self):
        self.data = np.c_[np.ones(len(self.data)) * -1 ,self.data]

    def start_training(self):
        print("starting training...")
        self.add_bias_to_data()
        self.generate_initial_weights()
        self.run_one_era()
        while self.adjusted_in_this_era:
            self.run_one_era()
        print("training ended! Convergence in {} era, {} adjustments made. ".format(self.eras_adjusting, self.adjustments_count))

    def run_one_era(self):
        self.adjusted_in_this_era = False
        for sample in self.data:
            #calc of summation
            sample_data = sample[:self.data_size]
            u = np.dot(self.weights, sample_data)
            y = self.activation_degree(u)
            y_ground_truth = sample[self.data_size]
            learning_and_error = self.learning_rate * (y - y_ground_truth)

            if learning_and_error > 0:
                delta = np.multiply(sample_data, learning_and_error)
                self.weights = np.add(self.weights, delta)
                self.adjusted_in_this_era = True
                self.adjustments_count += 1
                print("weight changed ! {}".format(self.weights))
        self.eras_adjusting +=1
        print("Era of count {}. Adjustments made so far {}".format(self.eras_adjusting, self.adjustments_count))

def import_numpy_data(path, size):
    raw_data = np.fromfile(file=path, dtype=float)
    formatted_data = np.array(np.split(raw_data, len(raw_data)/size))
    return formatted_data

def plot_scatter(title, xlabel, ylabel, plot_data, straight_line_p1 = 0, straight_line_p2 = 0, plot_line = False):
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    for k in plot_data:
        if k[2] == 0:
            plt.scatter(k[0], k[1], c='r')
        else:
            plt.scatter(k[0], k[1], c='b')
    if plot_line:
        x = plot_data[:,0]
        y = straight_line_p1 - straight_line_p2 * x
        plt.scatter(x, y, color="black")
    plt.show()

def obtain_line_by_weight(weights):
    factor1 = weights[0] / weights[2]
    factor2 = weights[1] / weights[2]
    return factor1, factor2

def benchmark_parameters(learn_rate, initial_weight_interval, data_to_bench):
    adj_qtd = []
    eras_qtd =[]
    for i in range(10):
        percep = RosenblattPerceptron (theta=0, learning_rate=learn_rate, data=data_to_bench, data_size=3, interval=initial_weight_interval )
        percep.learning_rate = learn_rate
        percep.data = data_to_bench
        percep.interval = initial_weight_interval
        percep.start_training()
        adj_qtd.append(percep.adjustments_count)
        eras_qtd.append(percep.eras_adjusting)
    return np.min(eras_qtd), np.mean(adj_qtd), np.std(adj_qtd)

<h1>2.1 Parte I - Resolvendo um problema Linearmente Separável</h1>
<h3>Lendo os dados do arquivo e plotando com as respectivas cores de classe<h3>

In [ ]:
filename = "dataAll.txt"
init_data = import_numpy_data(filename, 3)
plot_scatter("Data All", "x1", "x2", plot_data = init_data)

<h3>Efetuando o treinamento com theta 0, learning rate 0.1 e intervalo para os pesos de (-0.5, 0.5)</h3>

In [ ]:
perceptron = RosenblattPerceptron(theta=0, learning_rate=0.1, data=init_data, data_size=3, interval=[-0.5, 0.5])
perceptron.start_training()

<h3>Plotando o resultado, gerando a reta equação dos pesos obtidos</h3>

In [ ]:
p1, p2 = obtain_line_by_weight(perceptron.weights)
print("Quantidade de épocas do treinamento: {}, Quantidade de ajustes feitos nos pesos: {}".format(perceptron.eras_adjusting, perceptron.adjustments_count))
plot_scatter(title="Data All", xlabel="x1", ylabel="x2", plot_data=init_data, straight_line_p1 = p1, straight_line_p2 = p2, plot_line=True)

<h1>2.2 - Experimentação </h1>

<h3> Primeiro serão plotados os exemplos desta questão </h3>

In [ ]:
filename = "data0.txt"
data22 = import_numpy_data(filename, 3)
plot_scatter("Data 0", "x1", "x2", plot_data = data22)


In [ ]:
learn_rates_to_try = [0.4, 0.1, 0.01]
initial_weights_to_try = [[-100,100], [-0.5, 0.5]]
lr = learn_rates_to_try[0]
initial_weights_interval = initial_weights_to_try[0]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table = PrettyTable()
rna_table.title = "Configurações"
rna_table.field_names = ["Taxa de Aprendizado", "Intervalo de Pesos", "Quantidade de Ajustes", "Menor número de épocas para convergência"]

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])

In [ ]:
lr = learn_rates_to_try[0]
initial_weights_interval = initial_weights_to_try[1]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])


In [ ]:
lr = learn_rates_to_try[1]
initial_weights_interval = initial_weights_to_try[0]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])


In [ ]:
lr = learn_rates_to_try[1]
initial_weights_interval = initial_weights_to_try[1]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])

In [ ]:
lr = learn_rates_to_try[2]
initial_weights_interval = initial_weights_to_try[0]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])

In [ ]:
lr = learn_rates_to_try[2]
initial_weights_interval = initial_weights_to_try[1]
era_min, mean, std =  benchmark_parameters(lr, initial_weights_interval, data22)

In [ ]:
point1, point2 = obtain_line_by_weight(perceptron.weights)
print("Combination of learning rate {} with initial weight interval {}".format(lr, initial_weights_interval))
plot_scatter(title="Data 0", xlabel="x1", ylabel="x2", straight_line_p1 = point1, straight_line_p2 = point2, plot_data=data22, plot_line=True)

In [ ]:
rna_table.add_row(["n={}".format(lr),
                           "({},+{})".format(initial_weights_interval[0],initial_weights_interval[1]),
                           "{}+-{}".format(mean, std),
                           "{}".format(era_min)
                           ])


In [ ]:
print(rna_table)